## Evaluate Personalization in Retail Demo Store's Web UI

Now that you've enabled each personalization feature by setting the respective campaign ARN, you can test these
personalization features through the Web App UI. If you haven't already opened a browser window/tab to the Retail Demo Store Web UI,
navigate to the CloudFormation console in this AWS account and check the Outputs section of the stack used to launch
the Retail Demo Store. Make sure you're checking the base stack and not the nested stacks that were created.
In the Outputs section look for the output named: WebURL and browse to the URL provided.

![CloudFormation Outputs](../images/cfn-webui-outputs.png)

If you haven't already created a user account in your Retail Demo Store instance, let's create one now.
 Once you've accessed the Retail Demo Store Web UI, you can logon and create a new account.
 Click on the Sign In button and then the "**No account? Create account**" link to create an account.
  Follow the prompts and enter the required data. You will need to provide a valid email address 
  in order to receive an email with the confirmation code to validate your account.

Once you've created and validated your account, click on the Sign In button again and sign in with the account you created.

In [ ]:
%store -r

In [ ]:
# Import Dependencies

import boto3
import json
import time
from time import sleep
import random
import uuid

# Setup Clients

personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')
personalize_events = boto3.client('personalize-events')

servicediscovery = boto3.client('servicediscovery')
ssm = boto3.client('ssm')


# The Uid is a unique ID and we need it to find the role made by CloudFormation
with open('/opt/ml/metadata/resource-metadata.json') as f:
    data = json.load(f)
sagemaker = boto3.client('sagemaker')
sagemakerResponce = sagemaker.list_tags(ResourceArn=data["ResourceArn"])
for tag in sagemakerResponce["Tags"]:
    if tag['Key'] == 'Uid':
        Uid = tag['Value']
        break

print('Uid:', Uid)
print('Region:', ssm.meta.region_name)

### Emulate Shopper

To confirm product recommendations are personalized, you can emulate a different shopper.
Click on your username in the top right-corner and then select Switch Shoppers
[**Profile**](https://github.com/aws-samples/retail-demo-store/blob/master/src/web-ui/src/authenticated/Profile.vue).
You can have a shopper auto-selected for you or you can choose your own.
In the shopper selection window, specify an age range and a primary shopping interest. Click Submit and a closely matching shopper is shown, confirm your choice or try again.
Product recommendations should match the persona of the shopper you've selected.


### Viewing Related Product Recommendations

Let's start with the Related Product Recommendations use-case.
This campaign for this use-case is based on the [SIMS](https://docs.aws.amazon.com/personalize/latest/dg/native-recipe-sims.html)
recipe which uses item-to-item collaborative filtering at its core to derive an understanding of how users interact with similar items.

Browse to a [product detail page](https://github.com/aws-samples/retail-demo-store/blob/master/src/web-ui/src/public/ProductDetail.vue)
and evaluate the products listed in the **What other items do customers view related to this product?** section.
You should see the recipe name displayed below the section header.
This tells you that results are actually coming from the related item campaign.
If you don't see the recipe name, the page is using default behavior of displaying products from the same category
(verify that the campaign was created successfully above **and** the campaign ARN is set as an SSM parameter).

Given the shopper personas we used to generate historical data, do the related item recommendations make sense?
For example, given that one of the shopper personas used across many of the customers is "footwear_outdoors",
you should see related products from both of these categories when viewing a product from either category.
Although this is a somewhat contrived example, it does illustrate how Personalize understands an affinity
for products across these categories.

![Related Product Recommendations](./images/retaildemostore-related-products.jpg)

### Viewing Product Recommendations

With the user emulation saved, browse to the Retail Demo Store
[home page](https://github.com/aws-samples/retail-demo-store/blob/master/src/web-ui/src/public/Main.vue) and evaluate
the products listed in the **Inspired by your shopping trends** section (towards bottom of page).
Do they appear consistent with the shopping persona you're emulating? For the screenshots listed here,
the user was trained with historical data based on the "footwear_outdoors" persona so we should see product
recommendations from the Footwear and Outdoors categories.

![Personalized Product Recommendations](./images/retaildemostore-product-recs.jpg)

Note that if the section is titled **Featured** or you don't see the Personalize recipe name displayed, this
 indicates that either you are not signed in as a user or the campaign ARN is not set as the appropriate SSM parameter.
 Double check that the campaign was created successfully above and that the campaign ARN is set in SSM.

### Personalized Ranking

Finally, let's evaluate the personalizated ranking use-case.
There are two places where personalized ranking is implemented in the Retail Demo Store.
With a user emulated, browse to the featured product category list by clicking on "Featured" from the Retail Demo Store home page.
Note how for the emulated user with a persona of "footwear_outdoors" that the shoe is moved to the first product.
(See [CategoryDetail.vue](https://github.com/aws-samples/retail-demo-store/blob/master/src/web-ui/src/public/CategoryDetail.vue)).

![Personalized Product Ranking](./images/retaildemostore-personalized-ranking.jpg)

The other feature where personalized ranking is implemented is in
[search results](https://github.com/aws-samples/retail-demo-store/blob/master/src/web-ui/src/public/Search.vue).
Start typing a word in the search box and a search result widget will be displayed.
If the results were reranked by Personalize, you will see a "Personalize Ranking" annotation in the search box.
For the emulated user with a historical affinity for footwear and outdoors,
notice that a search for product keywords starting with "s" will move shoes to the top of the results.

![Personalized Search Results](./images/retaildemostore-personalized-search.jpg)

If the search functionality is not working at all for you, make sure that you
completed the [Search workshop](../0-StartHere/Search.ipynb).

### Personalized Discounts

The Personalized Discounts are enabled against the IVS demo tab ("Live") in the navigation bar where discounts are chosen over the
current set of products streamed from the IVS live stream:

![Personalized Discounts](./images/retaildemostore-personalized-discounts.png)

Currently 2 products are selected for each video to be offered a discount.


## Event Tracking - Keeping up with evolving user intent

Up to this point we have trained and deployed three Amazon Personalize campaigns based on historical data that we
 generated in this workshop. This allows us to make related product, user recommendations, and rerank product
  lists based on already observed behavior of our users. However, user intent often changes in real-time such
  that what products the user is interested in now may be different than what they were interested in a week ago,
  a day ago, or even a few minutes ago. Making recommendations that keep up with evolving user intent is one of the
   more difficult challenges with personalization. Fortunately, Amazon Personalize has a mechanism for this exact issue.

Amazon Personalize supports the ability to send real-time user events (i.e. clickstream) data into the service.
Personalize uses this event data to improve recommendations. It will also save these events and automatically
include them when solutions for the same dataset group are re-created (i.e. model retraining).

The Retail Demo Store's Web UI already has
[logic to send events](https://github.com/aws-samples/retail-demo-store/blob/master/src/web-ui/src/analytics/AnalyticsHandler.js)
such as 'ProductViewed', 'ProductAdded', 'OrderCompleted', and others as they occur in real-time to a Personalize Event Tracker.
These are the same event types we used to initially create the solutions and campaigns for our three use-cases.
All we need to do is create an event tracker in Personalize, set the tracking Id for the tracker in an SSM parameter,
and rebuild the Web UI service to pick up the change.

### Create Personalize Event Tracker

Let's start by creating an event tracker for our dataset group.

In [ ]:
event_tracker_response = personalize.create_event_tracker(
    datasetGroupArn="arn:aws:personalize:eu-west-1:943798077672:dataset-group/retaildemostore",
    name='retaildemostore-event-tracker'
)

event_tracker_arn = event_tracker_response['eventTrackerArn']
event_tracking_id = event_tracker_response['trackingId']

print('Event Tracker ARN: ' + event_tracker_arn)
print('Event Tracking ID: ' + event_tracking_id)

### Wait for Event Tracker Status to Become ACTIVE

The event tracker should take a minute or so to become active.

In [ ]:
status = None
max_time = time.time() + 60*60 # 1 hours
while time.time() < max_time:
    describe_event_tracker_response = personalize.describe_event_tracker(
        eventTrackerArn = event_tracker_arn
    )
    status = describe_event_tracker_response["eventTracker"]["status"]
    print("EventTracker: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(15)

### Update SSM Parameter To Enable Event Tracking

The Retail Demo Store's Web UI service just needs a Personalize Event Tracking Id to be able to send events to Personalize. The CodeBuild configuration for the Web UI service will pull the event tracking ID from an SSM parameter. 

Let's set our tracking ID in an SSM parameter.

In [ ]:
response = ssm.put_parameter(
    Name='retaildemostore-personalize-event-tracker-id',
    Description='Retail Demo Store Personalize Event Tracker ID Parameter',
    Value='{}'.format(event_tracking_id),
    Type='String',
    Overwrite=True
)

### Trigger Web UI Service Release

Next let's trigger a new release of the Retail Demo Store's Web UI service so that it will pick up our SSM parameter change.

In the AWS console, browse to the AWS Code Pipeline service. Find the pipeline with **WebUIPipeline** in the name. Click on the pipeline name.

![AWS CodePipeline](./images/retaildemostore-codepipeline.png)

#### Trigger Release

To manually trigger a new release, click the **Release change** button, click the **Release** button on the popup dialog window, and then wait for the pipeline to build and deploy. This will rebuild the web app, deploy it to the web UI S3 bucket, and invalidate the CloudFront distribution to force browsers to load from the origin rather than from their local cache.

![AWS CodePipeline Release](./images/retaildemostore-codepipeline-release.png)

### Verify Event Tracking

Return to your web browser tab/window where the Retail Demo Store Web UI is loaded and **reload the web app/page**. Reloading the page is important so that the web app is reloaded in your browser and the new event tracking configuration is loaded as well.

There are a couple ways to verify that events are being sent to the Event Tracker. First, you can use your browser's developer tools to monitor the network calls made by the Retail Demo Store Web UI when you're browsing to product detail pages, adding items to carts, and completing orders. The other way you can verify that events are being received by the event tracker is in CloudWatch metrics for Personalize.

1. If you have done so, **reload the web app by refreshing/reloading your browser page.**
2. If not already signed in as a storefront user, sign in as (or create) a user. 
3. In the Retail Demo Store Web app, view product detail pages, add items to your cart, complete an order.
4. Verify that the Web UI is making "events" calls to the Personalize Event Tracker.
5. In the AWS console, browse to CloudWatch and then Metrics.

![Personalize CloudWatch Metrics](./images/retaildemostore-eventtracker-cw.png)

If events are not being sent to the event tracker, make sure that the WebUIPipeline pipeline was built and deployed successfully and that you reloaded the web app in your browser.

To assess the impact of real-time event tracking in recommendations made by the user recommendations on the home page, follow these steps.

1. Sign in as (or create) a storefront user.
2. View the product recommendations displayed on the home page under the "Inspired by your shopping trends" header. Take note of the products being recommended.
3. View products from categories that are not being recommended by clicking on their "Details" button. When you view the details for a product, an event is fired and sent to the Personalize event tracker.
4. Return to the home page and you should see products being recommended that are the same or similar to the ones you just viewed.

### Cold User Recommendations

One of the key features of Personalize is being able to cold start users. Cold users are typically those who are new to your site or application and cold starting a user is getting from no personalization to making personalized recommendations in real-time. 

Personalize accomplishes cold starting users via the Event Tracker, just as we saw above with existing users. However, since new users are typically anonymous for a period of time before they create an account or may choose to transact as a guest, personalization is a valuable tool to help convert those anonymous users to transacting users. 

The challenge here is that Personalize needs a `userId` for anonymous users before it can make personalized recommendations. The Retail Demo Store solves this challenge by creating a provisional user ID the moment an anonymous user first hits the site. This provisional user ID is then used when streaming events to the Event Tracker and when retrieving recommendations from the Recommendations service. This allows the Retail Demo Store to start serving personalized recommendations after the first couple events are streamed to Personalize. Before recommendations can be personalized, Personalize will provide recommendations for popular items as a fallback.

To see this behavior in action, browse to the Retail Demo Store storefront using a different browser, an Incognito/Private window, or sign out of your existing account. What you should see on the home page is that instead of **"Inspired by your shopping behavior"**, the section is **"Trending products"**. After you click on a couple provide detail pages, return to the home page and see that the section title and recommendations have changed. This indicates that recommendations are now being personalized and will continue to become more relevant as you engage with products.

Similarly, the category pages will rerank products at first based on popularity and then become more and more personalized.

There are some challenges with this approach, though. First is the question of what to do with the provisional user ID when the user creates an account. To maintain continuity of the user's interaction history, the Retail Demo Store passes the provisional user ID to the Users microservice when creating a new user account. The Users service then uses this ID as the user's ID going forward. Another challenge is how to handle a user that anonymously browses the site using multiple devices such as on the mobile device and then on a desktop/laptop. In this case, separate provisional user IDs are generated for sessions on each device. However, once the user creates an account on one device and then signs in with that account on the other device, both devices will starting using the same user ID going forward. A side effect here is that the interaction history from one of the devices will be orphaned. This is an acceptable tradeoff given the benefit of cold starting users earlier and is functionally the same UX without this scheme. Additional logic could be added to merge the interaction history from both prior anonymous sessions when the user creates an account. Also, customer data platforms can be used to help manage this for you.

## Create Purchased Products Filter

Amazon Personalize supports the ability to create [filters](https://docs.aws.amazon.com/personalize/latest/dg/filter.html) that can be used to exclude items from being recommended that meet a filter expression. Since it's a poor user experience to recommend products that a user has already purchased, we will create a filter that excludes recently purchased products. We'll do this by creating a filter expression that excludes items that have an interaction with an event type of `OrderCompleted` for the user.

> As noted above, the Retail Demo Store web application streams clickstream events to Personalize when the user performs various actions such as viewing and purchasing products. The filter created below allows us to use those events as exclusion criteria. See the [AnalyticsHandler.js](https://github.com/aws-samples/retail-demo-store/blob/master/src/web-ui/src/analytics/AnalyticsHandler.js) file for the code that sends clickstream events.

In [ ]:
response = personalize.create_filter(
    name = 'retaildemostore-filter-purchased-products',
    datasetGroupArn = "arn:aws:personalize:eu-west-1:943798077672:dataset-group/retaildemostore",
    filterExpression = 'EXCLUDE itemId WHERE INTERACTIONS.event_type in ("OrderCompleted")'
)
 
filter_arn = response['filterArn']
print(f'Filter ARN: {filter_arn}')

### Wait for Filter Status to Become ACTIVE

The filter should take a minute or so to become active.

In [ ]:
status = None
max_time = time.time() + 60*60 # 1 hours
while time.time() < max_time:
    describe_filter_response = personalize.describe_filter(
        filterArn = filter_arn
    )
    status = describe_filter_response["filter"]["status"]
    print("Filter: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(15)

### Test Purchased Products Filter

To test our purchased products filter, we will request recommendations for a random user. Then we will send an `OrderCompleted` event for one of the recommended products to Personalize using the event tracker created above. Finally, we will request recommendations again for the same user but this time specify our filter.

In [ ]:
# Pick a user ID in the range of test users and fetch 5 recommendations.
user_id = '456'
get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = "arn:aws:personalize:eu-west-1:943798077672:campaign/retaildemostore-product-personalization",
    userId = user_id,
    numResults = 5
)

item_list = get_recommendations_response['itemList']
print(json.dumps(item_list, indent=2))

Next let's randomly select an item from the returned list of recommendations to be our product to purchase.

In [ ]:
product_id_to_purchase = random.choice(item_list)['itemId']
print(f'Product to simulate purchasing: {product_id_to_purchase}')

Next let's send an `OrderCompleted` event to Personalize to simulate that the product was just purchased.
This will match the criteria for our filter.
In the Retail Demo Store web application, this event is sent for each product in the order after the order is completed.

In [ ]:
response = personalize_events.put_events(
    trackingId = event_tracking_id,
    userId = user_id,
    sessionId = str(uuid.uuid4()),
    eventList = [
        {
            'eventId': str(uuid.uuid4()),
            'eventType': 'OrderCompleted',
            'itemId': str(product_id_to_purchase),
            'sentAt': int(time.time()),
            'properties': '{"discount": "No"}'
        }
    ]
)

# Wait for OrderCompleted event to become consistent.
time.sleep(5)

print(json.dumps(response, indent=2))

Finally, let's retrieve recommendations for the user again but this time specifying the filter to exclude recently
purchased items. We do this by passing the filter's ARN via the `filterArn` parameter.
In the Retail Demo Store, this is done in the
[Recommendations](https://github.com/aws-samples/retail-demo-store/tree/master/src/recommendations) service.

In [ ]:
get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = "arn:aws:personalize:eu-west-1:943798077672:campaign/retaildemostore-product-personalization",
    userId = user_id,
    numResults = 5,
    filterArn = filter_arn
)

item_list = get_recommendations_response['itemList']
print(json.dumps(item_list, indent=2))

The following code will raise an assertion error if the product we just purchased is still recommended.

In [ ]:
found_item = next((item for item in item_list if item['itemId'] == product_id_to_purchase), None)
if found_item:
    assert found_item == False, 'Purchased item found unexpectedly in recommendations'
else:
    print('Purchased item filtered from recommendations for user!')

### Update Filter SSM Parameter

With our filter created and tested, the last step is to update the SSM parameter that is used throughout the
Retail Demo Store project to detect the filter ARN.

The [Recommendations](https://github.com/aws-samples/retail-demo-store/tree/master/src/recommendations) service already has logic to look for the purchased products filter ARN in SSM and use it when fetching recommendations. All we have to do is set the filter's ARN in SSM.

In [ ]:
response = ssm.put_parameter(
    Name='retaildemostore-personalize-filter-purchased-arn',
    Description='Retail Demo Store Personalize Filter Purchased Products Arn Parameter',
    Value='{}'.format(filter_arn),
    Type='String',
    Overwrite=True
)

Now if you test completing an order for one or more items in the Retail Demo Store web application for a user,
those products should no longer be included in recommendations for that user.
Test it out by purchasing a recommended product from the "Inspired by your shopping trends"
section of the home page and then verifying that the product is no longer recommended.

## Workshop Complete

Congratulations! You have completed the Retail Demo Store Personalization Workshop.

### Cleanup

If you launched the Retail Demo Store in your personal AWS account **AND** you're done with all workshops, you can follow the [Personalize workshop cleanup](./1.3-Personalize-Cleanup.ipynb) notebook to delete all of the Amazon Personalize resources created by this workshop. **IMPORTANT: since the Personalize resources were created by this notebook and not CloudFormation, deleting the CloudFormation stack for the Retail Demo Store will not remove the Personalize resources. You MUST run the [Personalize workshop cleanup](./1.3-Personalize-Cleanup.ipynb) notebook or manually clean up these resources.**

If you are participating in an AWS managed event such as a workshop and using an AWS provided temporary account, you can skip the cleanup workshop unless otherwise instructed.